In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_addons.layers import ESN
from kerastuner.tuners import RandomSearch, BayesianOptimization

Using TensorFlow backend


/Users/florianwicher/miniforge3/envs/tf_test/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/var/folders/24/dwryd6_d22d4mw0dwwkwbcjh0000gn/T/ipykernel_2092/2778276001.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, BayesianOptimization


In [2]:
data = pd.read_parquet('train_data_sensor_fault.parquet.gzip').to_numpy().astype(np.float32)


data = data[:int(0.2 * len(data))]
data.shape

(1478304, 31)

In [3]:
#choose  sensor for hp search

sensor_id = 15

inputs_idx = list(range(data.shape[1]));inputs_idx.remove(sensor_id)


x_train, y_train = data[:,inputs_idx], data[:,sensor_id]

x = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)
y = np.array(y_train).reshape(y_train.shape[0],1)

In [4]:
dense_activations = ['tanh', 'sigmoid', 'linear', 'relu']
esn_activations = ['tanh', 'sigmoid']

def create_model(hp):
    model = keras.Sequential()
    model.add(layers.GRU(31, activation = hp.Choice('ESN_1_activation', esn_activations), return_sequences=True))
    model.add(layers.Dropout(hp.Choice('Dropout_rate', [.1,.2,.3,.4,])))
    
    model.add(layers.GRU(31, activation = hp.Choice('ESN_2_activation', esn_activations), return_sequences=True))
    model.add(layers.Dropout(hp.Choice('Dropout_1_rate', [.1,.2,.3,.4,])))
    
    model.add(layers.GRU(31, activation = hp.Choice('ESN_3_activation', esn_activations), return_sequences=True))
    model.add(layers.Dropout(hp.Choice('Dropout_2_rate', [.1,.2,.3,.4,])))
    
    model.add(layers.GRU(31, activation = hp.Choice('ESN_4_activation', esn_activations), return_sequences=False))
    
    
    model.add(layers.Dense(hp.Int('Dense_1_units', min_value=5, max_value=31, step=2), hp.Choice('Dense_1_activation', dense_activations)))
    model.add(layers.Dense(hp.Int('Dense_2_units', min_value=5, max_value=31, step=2), hp.Choice('Dense_2_activation', dense_activations)))
    model.add(layers.Dense(hp.Int('Dense_3_units', min_value=5, max_value=31, step=2), hp.Choice('Dense_3_activation', dense_activations)))
    model.add(layers.Dense(1))
    
    model.compile(loss = 'mean_absolute_error',
                   optimizer ='adam',
                   metrics=['mean_absolute_error'])
    return model

In [5]:
tuner = BayesianOptimization(
    create_model,
    objective='val_mean_absolute_error',
    max_trials=20)

2023-10-18 12:55:55.042823: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-10-18 12:55:55.042840: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-18 12:55:55.042845: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-18 12:55:55.042871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-18 12:55:55.042882: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
with tf.device('/cpu:0'):
    tuner.search(x, y, batch_size=64, epochs=2, validation_split=0.2)


Trial 2 Complete [00h 13m 50s]
val_mean_absolute_error: 0.10324244946241379

Best val_mean_absolute_error So Far: 0.10324244946241379
Total elapsed time: 00h 29m 09s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |ESN_1_activation
0.3               |0.3               |Dropout_rate
tanh              |tanh              |ESN_2_activation
0.3               |0.2               |Dropout_1_rate
tanh              |tanh              |ESN_3_activation
0.2               |0.2               |Dropout_2_rate
tanh              |tanh              |ESN_4_activation
27                |21                |Dense_1_units
linear            |sigmoid           |Dense_1_activation
23                |11                |Dense_2_units
tanh              |tanh              |Dense_2_activation
27                |9                 |Dense_3_units
linear            |tanh              |Dense_3_activation

Epoch 1/2
  219/18479 [..........................

In [ ]:
best_hyper_parameters = tuner.get_best_hyperparameters()[0]
best_hyper_parameters.values

In [1]:
import numpy as np

In [6]:
a = np.array([1., 3., 8,])

b = np.array([3., 5., 9,])

c = np.abs(np.subtract(a,b))
c

array([2., 2., 1.])